# LOAD NECESSARY LIBRARIES

In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor

from pathlib import Path
import torch
import os
os.environ["HF_TOKEN"] = '<HF_TOKEN>'


import glob
from pathlib import Path
import PIL
import requests
from PIL import Image

import matplotlib.patches as patches
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt

import json
from IPython.display import clear_output

COLORS = [k  for  k in  mcolors.CSS4_COLORS.keys()]


/home/jovyan/bobrin/miniconda3/envs/GroundingMaira2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
 !pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:

import pandas as pd

In [5]:
!nvidia-smi

Tue Dec 17 15:11:08 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   31C    P0    66W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [6]:

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

# LOAD MODEL

In [7]:
model = AutoModelForCausalLM.from_pretrained("microsoft/maira-2", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/maira-2", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 6/6 [00:00<00:00,  8.47it/s]


In [8]:
device = torch.device("cuda")
model = model.eval()
model = model.to(device)

# LOAD DATASET

In [9]:
gold_500 = pd.read_csv('gold_500.csv')

In [10]:
gold_500

,Unnamed: 0,image_id,x1,y1,x2,y2,width,height,bbox_name,annot_id,original_x1,original_x2,original_y1,original_y2,original_width,original_height,annotator,coord224,coord_original,image_id_jpg
0,0,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.dcm,124.0,85.0,145.0,105.0,21.0,20.0,aortic arch,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.d...,1691.0,1978.0,900.0,1173.0,287.0,273.0,C&&W,"[124, 85, 145, 105]","[1691, 900, 1978, 1173]",00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.jpg
1,1,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.dcm,99.0,110.0,188.5,181.0,89.5,71.0,cardiac silhouette,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.d...,1350.5,2571.0,1241.0,2210.0,1220.5,969.0,C&&W,"[99, 110, 188, 181]","[1350, 1241, 2571, 2210]",00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.jpg
2,2,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.dcm,115.0,102.0,125.0,112.0,10.0,10.0,carina,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.d...,1568.0,1705.0,1132.0,1268.0,137.0,136.0,C&&W,"[115, 102, 125, 112]","[1568, 1132, 1705, 1268]",00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.jpg
3,3,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.dcm,98.0,110.5,117.5,131.5,19.5,21.0,cavoatrial junction,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.d...,1336.5,1602.5,1247.5,1534.5,266.0,287.0,C&&W,"[98, 110, 117, 131]","[1336, 1247, 1602, 1534]",00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.jpg
4,4,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.dcm,127.0,44.0,180.0,76.0,53.0,32.0,left apical zone,00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.d...,1732.0,2455.0,341.0,777.0,723.0,436.0,C&&W,"[127, 44, 180, 76]","[1732, 341, 2455, 777]",00046130-fd952ef0-57f2948d-491a16b4-5db3a18c.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12989,12989,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.dcm,33.0,80.0,99.0,118.0,66.0,38.0,right mid lung zone,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.d...,191.0,1091.0,1091.0,1609.0,900.0,518.0,C&&W,"[33, 80, 99, 118]","[191, 1091, 1091, 1609]",84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.jpg
12990,12990,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.dcm,38.0,25.0,104.0,80.0,66.0,55.0,right upper lung zone,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.d...,259.0,1159.0,341.0,1091.0,900.0,750.0,C&&W,"[38, 25, 104, 80]","[259, 341, 1159, 1091]",84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.jpg
12991,12991,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.dcm,95.0,63.0,112.0,101.0,17.0,38.0,svc,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.d...,1036.0,1268.0,859.0,1377.0,232.0,518.0,C&&W,"[95, 63, 112, 101]","[1036, 859, 1268, 1377]",84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.jpg
12992,12992,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.dcm,88.0,3.0,118.0,102.0,30.0,99.0,trachea,84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.d...,941.0,1350.0,40.0,1391.0,409.0,1351.0,C&&W,"[88, 3, 118, 102]","[941, 40, 1350, 1391]",84c38cc6-94deef69-e9407547-a24011a8-eac1cedd.jpg


In [11]:
# directory with 27 organ directories, where all the images are
!mkdir "<YOUR_NAME>" 

In [12]:
def get_sample_data( image_path, part):
  
    '''
      Parameters
      ----------
    image_path - the absolute path, where the image is
    part - the body part we need to detect
    '''
    def download_and_open(image_path) -> Image.Image:
#         response = requests.get(url, headers={"User-Agent": "MAIRA-2"}, stream=True) 
        return Image.open(image_path).resize((500, 500))

    frontal_image = download_and_open(image_path)
    name = image_path.split("/")[-1]


    sample_data = {
        "frontal": frontal_image,
        "technique": "PA view of the chest.",
        "phrase": gold_500.bbox_name[((gold_500.image_id_jpg==name) & (gold_500.bbox_name ==part))].values[0]  # For the phrase grounding example. This patient has pleural effusion.
    }
    return sample_data


In [13]:
gold_500 = pd.read_csv('gold_500.csv')
body_parts = gold_500.bbox_name.unique()
body_parts

array(['aortic arch', 'cardiac silhouette', 'carina',
       'cavoatrial junction', 'left apical zone', 'left clavicle',
       'left costophrenic angle', 'left hemidiaphragm',
       'left hilar structures', 'left lower lung zone', 'left lung',
       'left mid lung zone', 'left upper lung zone', 'right apical zone',
       'right atrium', 'right clavicle', 'right costophrenic angle',
       'right hemidiaphragm', 'right hilar structures',
       'right lower lung zone', 'right lung', 'right mid lung zone',
       'right upper lung zone', 'svc', 'trachea', 'upper mediastinum'],
      dtype=object)

In [14]:
# the structure of thisdictionary is part--> "{image_name --> coordinates of the bounding box}"

#this one is for the original body box
bboxes_orig = {} 
for part in body_parts:
    bboxes_orig[part] = {}

#this one is for the maira predicted body box
bboxes_maira = {} 
for part in body_parts:
    bboxes_maira[part] = {}


In [15]:
# the structure of thisdictionary is part--> "{image_name --> iou of the predicted bbox relating to original bbox}"
bboxes_iou_maira = {} 
for part in body_parts:
    bboxes_iou_maira[part] = {}

In [19]:
def adjust_box_for_original_image_size(box, width, height):
    """
    Parameters
    ----------
    box - predicted float [0,1] bbox
    width, height - width, height of the image
    
    this function returns the transformed int coordinates of the predicted bbox
    """

    crop_width = crop_height = min(width, height)
    x_offset = (width - crop_width) // 2
    y_offset = (height - crop_height) // 2

    norm_x_min, norm_y_min, norm_x_max, norm_y_max = box

    abs_x_min = int(norm_x_min *crop_width + x_offset)
    abs_x_max = int(norm_x_max *crop_width + x_offset)
    abs_y_min = int(norm_y_min * crop_height + y_offset)
    abs_y_max = int(norm_y_max * crop_height + y_offset)


    return (abs_x_min, abs_y_min, abs_x_max-abs_x_min, abs_y_max-abs_y_min)

In [18]:
def calculate_iou_mask(predictions, targets, epsilon=1e-7):
    predictions = predictions.byte()
    targets = targets.byte()
    intersection = (predictions & targets).sum((0, 1))
    union = (predictions | targets).sum((0, 1))
    iou = (intersection + epsilon) / (union + epsilon)
    return iou



In [17]:
from tqdm import tqdm

In [1]:
for part in body_parts:
    clear_output(wait=True)
    parent_dir = "<YOUR_NAME>"
    path = os.path.join(parent_dir, part) 
    os.mkdir(path) 

    
    for image in tqdm(gold_500.image_id_jpg.unique()):
        path1 = "<PATH TO MIMIC_500>"
        
        img_path = path1 + image
        
        if (os.path.exists(img_path)):
            
            
            if len(gold_500.original_x1[((gold_500.image_id_jpg==image) & (gold_500.bbox_name ==part))].values) > 0:
                W, H = Image.open(img_path).size
                img_try = Image.open(img_path).convert("RGB").resize((500, 500 ))
                x = gold_500.original_x1[((gold_500.image_id_jpg==image) & (gold_500.bbox_name ==part))].values[0]
                y = gold_500.original_y1[((gold_500.image_id_jpg==image) & (gold_500.bbox_name ==part))].values[0]
                w = gold_500.original_width[((gold_500.image_id_jpg==image) & (gold_500.bbox_name ==part))].values[0]
                h = gold_500.original_height[((gold_500.image_id_jpg==image) & (gold_500.bbox_name ==part))].values[0]
                bbox = [round(x / W *500) , round(y / H *500), round(w / W *500), round(h / H *500)]
                
                sample_data = get_sample_data(img_path, part)

                processed_inputs = processor.format_and_preprocess_phrase_grounding_input(
                    frontal_image=sample_data["frontal"],
                    phrase=sample_data["phrase"],
                    return_tensors="pt",
                )
                processed_inputs = processed_inputs.to(device)
                with torch.no_grad():
                    output_decoding = model.generate(
                        **processed_inputs,
                        max_new_tokens=150,
                        use_cache=True,
                    )
                prompt_length = processed_inputs["input_ids"].shape[-1]
                decoded_text = processor.decode(output_decoding[0][prompt_length:], skip_special_tokens=True)
                try:
                    prediction = processor.convert_output_to_plaintext_or_grounded_sequence(decoded_text)
                    coords = prediction[0][1]
                    if coords != None:
                        x,y,x1,y1 = coords[0]
                        
                        x_p, y_p, w_p, h_p =  adjust_box_for_original_image_size([x,y,x1,y1], 500, 500)
                        bboxes_maira[part][image] = (x_p, y_p, w_p, h_p)

                        x_o, y_o, w_o, h_o = bbox
                        bboxes_orig[part][image] = (x_o, y_o, w_o, h_o) 

                
                        fig, ax = plt.subplots()
                        ax.imshow(np.flipud(Image.open(img_path).resize((500, 500))), origin='lower')
            
                        rect1 = patches.Rectangle((x_p, 500- y_p), w_p, -h_p, linewidth=3, edgecolor='red', facecolor='none')
                        ax.add_patch(rect1)

               
                        rect2 = patches.Rectangle((x_o, 500- y_o), w_o, -h_o, linewidth=3, edgecolor='yellow', facecolor='none')
                        ax.add_patch(rect2)
                        
            
                        plt.show()
                        path_image = os.path.join(path, image) 
                        fig.savefig(path_image)
                        
                        W,H = 500, 500 
                        
                        mask_pred = torch.tensor(np.zeros((W,H)))
                        mask_pred[ x_p: x_p+w_p, y_p:y_p+h_p] = 1
                
                        mask_orig = torch.tensor(np.zeros((W,H)))
                        mask_orig[ x_o: x_o+w_o, y_o:y_o+h_o] = 1
                
                
                        bboxes_iou_maira[part][image] = calculate_iou_mask(mask_pred, mask_orig).item()
#                    
                    else:
                        print("this part is ok", prediction)
                except (AssertionError, ValueError):
                    pass
            else:
                pass
        else:
            pass
    with open('<NAME>.json', 'w') as json_file:
        json.dump(bboxes_maira, json_file, allow_nan=True) # maira predicted bboxes

    with open('<NAME>.json', 'w') as json_file:
        json.dump(bboxes_orig, json_file, allow_nan=True) # ground truth bboxes

    with open('<NAME>.json', 'w') as json_file:
        json.dump(bboxes_iou_maira, json_file, allow_nan=True) #maira iou
        